# 10. 이미지 변형

## 흑백

이미지를 흑백으로 읽음

In [1]:
import cv2

img = cv2.imread("images/img.jpg", cv2.IMREAD_GRAYSCALE)

cv2.imshow("img", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

불러온 이미지를 흑백으로 변경

In [2]:
import cv2

img = cv2.imread("images/img.jpg")
dst = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow("img", img)
cv2.imshow("gray", dst)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 흐림

### 가우시안 블러

커널 사이즈 변화에 따른 흐림

In [1]:
import cv2

img = cv2.imread("images/img.jpg")

# (3, 3), (5, 5), (7, 7)
kernel_3 = cv2.GaussianBlur(img, (3, 3), 0)
kernel_5 = cv2.GaussianBlur(img, (5, 5), 0)
kernel_7 = cv2.GaussianBlur(img, (7, 7), 0)


cv2.imshow("img", img)
cv2.imshow("kernel_3", kernel_3)
cv2.imshow("kernel_5", kernel_5)
cv2.imshow("kernel_7", kernel_7)

cv2.waitKey(0)
cv2.destroyAllWindows()

표준편차 변화에 따른 흐림

In [3]:
import cv2

img = cv2.imread("images/img.jpg")

sigma_1 = cv2.GaussianBlur(img, (0, 0), 1)  # sigmaX: 가우시안 커널의 x 방향의 표준편차
sigma_2 = cv2.GaussianBlur(img, (0, 0), 2)
sigma_3 = cv2.GaussianBlur(img, (0, 0), 3)

cv2.imshow("img", img)
cv2.imshow("sigma_1", sigma_1)
cv2.imshow("sigma_2", sigma_2)
cv2.imshow("sigma_3", sigma_3)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 원근

사다리꼴 이미지 펼치기

In [9]:
import cv2
import numpy as np

img = cv2.imread("images/newspaper.jpg")

width, height = 640, 350  # 가로 크기 640, 세로 크기 350 으로 결과물 출력
# 좌상, 우상, 우하, 좌하 (시계 방향으로 4 지점 정의)
src = np.array(
    [
        [511, 352],
        [1008, 345],
        [1122, 594],
        [455, 604],
    ],
    dtype=np.float32,
)  # input 4개 지점
dst = np.array(
    [
        [0, 0],
        [width, 0],
        [width, height],
        [0, height],
    ],
    dtype=np.float32,
)  # output 4개 지점

matrix = cv2.getPerspectiveTransform(src, dst)  # Matrix 얻어옴
result = cv2.warpPerspective(img, matrix, (width, height))  # matrix 대로 변환을 함

cv2.imshow("img", img)
cv2.imshow("result", result)

cv2.waitKey(0)
cv2.destroyAllWindows()

회전된 이미지 올바로 세우기

In [17]:
import cv2
import numpy as np

img = cv2.imread("images/poker.jpg")

width, height = 530, 710
# 좌상, 우상, 우하, 좌하 (시계 방향으로 4 지점 정의)
src = np.array(
    [
        [705, 137],
        [1133, 414],
        [726, 1007],
        [276, 700],
    ],
    dtype=np.float32,
)  # input 4개 지점
dst = np.array(
    [
        [0, 0],
        [width, 0],
        [width, height],
        [0, height],
    ],
    dtype=np.float32,
)  # output 4개 지점

matrix = cv2.getPerspectiveTransform(src, dst)  # Matrix 얻어옴
result = cv2.warpPerspective(img, matrix, (width, height))  # matrix 대로 변환을 함

cv2.imshow("img", img)
cv2.imshow("result", result)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 미니 프로젝트: 반자동 문서 스캐너

### 마우스 이벤트 등록

In [10]:
import cv2


def mouse_handler(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # 마우스 왼쪽 버튼 Down
        print("왼쪽 버튼 Down")
        print(x, y)
    elif event == cv2.EVENT_LBUTTONUP:  # 마우스 왼쪽 버튼 Up
        print("왼쪽 버튼 Up")
        print(x, y)
    elif event == cv2.EVENT_LBUTTONDBLCLK:  # 마우스 왼쪽 버튼 더블 클릭
        print("왼쪽 버튼 Double Click")
    # elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 이동
    #     print("마우스 이동")
    elif event == cv2.EVENT_RBUTTONDOWN:  # 오른쪽 버튼 Down
        print("오른쪽 버튼 Down")


img = cv2.imread("images/poker.jpg")

cv2.namedWindow("img")  # img 란 이름의 윈도우를 먼저 만들어두는 것. 여기에 마우스 이벤트를 처리하기 위한 핸들러 적용
cv2.setMouseCallback("img", mouse_handler)

cv2.imshow("img", img)

cv2.waitKey(0)
cv2.destroyAllWindows()

왼쪽 버튼 Down
572 339
왼쪽 버튼 Up
917 485
왼쪽 버튼 Down
677 660
왼쪽 버튼 Up
677 660
왼쪽 버튼 Double Click
왼쪽 버튼 Up
677 660


### 프로젝트

In [34]:
import cv2
import numpy as np

src_img = cv2.imread("images/poker.jpg")
point_list = []
drawing = False  # 선을 그릴지 여부

COLOR = (255, 0, 255)  # Magenta
THICKNESS = 3


def mouse_handler(event, x, y, flags, param):
    global drawing
    dst_img = src_img.copy()

    # 영역 꼭짓점 선택
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True  # 선을 그리기 시작
        point_list.append((x, y))

    # 영역 선택 초기화
    if event == cv2.EVENT_RBUTTONDOWN:
        point_list.clear()
        drawing = False

    if drawing:
        prev_point = None  # 직선의 시작점
        for point in point_list:
            cv2.circle(dst_img, point, 15, COLOR, cv2.FILLED)
            if prev_point:
                cv2.line(dst_img, prev_point, point, COLOR, THICKNESS, cv2.LINE_AA)
            prev_point = point

        next_point = (x, y)
        if len(point_list) >= 4:
            next_point = point_list[0]  # 첫 번째 클릭한 지점
            show_result()  # 결과 출력
        cv2.line(dst_img, prev_point, next_point, COLOR, THICKNESS, cv2.LINE_AA)

    cv2.imshow("img", dst_img)


def show_result():
    width, height = 530, 710
    src = np.float32(point_list[:4])
    dst = np.array(
        [
            [0, 0],
            [width, 0],
            [width, height],
            [0, height],
        ],
        dtype=np.float32,
    )

    matrix = cv2.getPerspectiveTransform(src, dst)
    result = cv2.warpPerspective(src_img, matrix, (width, height))
    cv2.imshow("result", result)


cv2.namedWindow("img")  # img 란 이름의 윈도우를 먼저 만들어두는 것. 여기에 마우스 이벤트를 처리하기 위한 핸들러 적용
cv2.setMouseCallback("img", mouse_handler)

cv2.imshow("img", src_img)

cv2.waitKey(0)
cv2.destroyAllWindows()